In [101]:
from pandas_gbq import read_gbq
from google.oauth2 import service_account
import pandas as pd
from pathlib import Path

download_dir = Path().parent.resolve()
credentials_path = f"{str(download_dir)}/credentials.json"

credentials = service_account.Credentials.from_service_account_file(credentials_path)

In [108]:
query = """
SELECT *
FROM `ps-eng-dados-ds3x.anaflavia_199.icf_icc_refined`
LIMIT 12
"""

project_id = 'ps-eng-dados-ds3x'

df = read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df

C:\Users\anafl\Documents\ds3x_\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,ÍNDICES E SEGMENTAÇÕES,variacao_percentual,load_timestamp
0,Índice das Condições Econômicas Atuais,260.8,2025-01-22 02:40:05.316468+00:00
1,ICF,-166.2,2025-01-22 02:40:05.316468+00:00
2,Índice de Confiança do Consumidor,-123.2,2025-01-22 02:40:05.316468+00:00
3,Índice de Expectativas do Consumidor,-81.4,2025-01-22 02:40:05.316468+00:00


In [47]:
query = """
CREATE OR REPLACE TABLE `ps-eng-dados-ds3x.anaflavia_199.icc_trusted` AS
SELECT DISTINCT *
FROM `ps-eng-dados-ds3x.anaflavia_199.icc_raw`
"""

# Execute a consulta SQL para criar a nova tabela no BigQuery
from google.cloud import bigquery

# Crie um cliente do BigQuery usando as credenciais
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Execute a consulta
query_job = client.query(query)

# Aguarde a conclusão da consulta
query_job.result()

print("Tabela 'icc_trusted' criada com sucesso no BigQuery!")


Tabela 'icc_trusted' criada com sucesso no BigQuery!


In [48]:
query = """
CREATE OR REPLACE TABLE `ps-eng-dados-ds3x.anaflavia_199.icf_trusted` AS
SELECT DISTINCT *
FROM `ps-eng-dados-ds3x.anaflavia_199.icf_raw`
"""

# Execute a consulta SQL para criar a nova tabela no BigQuery
from google.cloud import bigquery

# Crie um cliente do BigQuery usando as credenciais
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Execute a consulta
query_job = client.query(query)

# Aguarde a conclusão da consulta
query_job.result()

print("Tabela 'icf_trusted' criada com sucesso no BigQuery!")

Tabela 'icf_trusted' criada com sucesso no BigQuery!


In [49]:
# Verifique as colunas da tabela
table_ref = client.dataset('anaflavia_199').table('icc_trusted')
schema = client.get_table(table_ref).schema

# Listar os nomes das colunas
columns = [field.name for field in schema]
print(columns)

['ÍNDICES E SEGMENTAÇÕES', '12_2023', '01_2024', '02_2024', '03_2024', '04_2024', '05_2024', '06_2024', '07_2024', '08_2024', '09_2024', '10_2024', '11_2024', '12_2024', 'dez-24_nov-24', 'dez-24_dez-23', 'load_timestamp']


In [50]:
# Verifique as colunas da tabela
table_ref = client.dataset('anaflavia_199').table('icf_trusted')
schema = client.get_table(table_ref).schema

# Listar os nomes das colunas
columns = [field.name for field in schema]
print(columns)

['ÍNDICES E SEGMENTAÇÕES', '12_2023', '01_2024', '02_2024', '03_2024', '04_2024', '05_2024', '06_2024', '07_2024', '08_2024', '09_2024', '10_2024', '11_2024', '12_2024', 'dez-24_nov-24', 'dez-24_dez-23', 'load_timestamp']


In [109]:
query = """
CREATE OR REPLACE TABLE `ps-eng-dados-ds3x.anaflavia_199.icf_icc_refined` AS
WITH icc_data AS (
    SELECT 
        icc.`ÍNDICES E SEGMENTAÇÕES`,
        ROUND(((icc.`dez-24_nov-24` - icc.`dez-24_dez-23`) / icc.`dez-24_dez-23`) * 100, 1) AS variacao_percentual,
        CURRENT_TIMESTAMP() AS load_timestamp
    FROM 
        `ps-eng-dados-ds3x.anaflavia_199.icc_trusted` icc
    WHERE 
        icc.`ÍNDICES E SEGMENTAÇÕES` IN ('Índice de Confiança do Consumidor', 'Índice das Condições Econômicas Atuais', 'Índice de Expectativas do Consumidor')
),
icf_data AS (
    SELECT 
        icf.`ÍNDICES E SEGMENTAÇÕES`,
        ROUND(((icf.`dez-24_nov-24` - icf.`dez-24_dez-23`) / icf.`dez-24_dez-23`) * 100, 1) AS variacao_percentual,
        CURRENT_TIMESTAMP() AS load_timestamp
    FROM 
        `ps-eng-dados-ds3x.anaflavia_199.icf_trusted` icf
    WHERE 
        icf.`ÍNDICES E SEGMENTAÇÕES` = 'ICF'
)

SELECT * FROM icc_data
UNION ALL
SELECT * FROM icf_data
"""
project_id = 'ps-eng-dados-ds3x'

df = read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)
df


C:\Users\anafl\Documents\ds3x_\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,ÍNDICES E SEGMENTAÇÕES,variacao_percentual,load_timestamp


GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/ps-eng-dados-ds3x/queries?prettyPrint=false: Access Denied: Table ps-eng-dados-ds3x:seu_dataset.sua_tabela: User does not have permission to query table ps-eng-dados-ds3x:seu_dataset.sua_tabela, or perhaps it does not exist.